In [1]:
pip install transformers datasets seqeval

import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset
import pandas as pd

# Load pre-trained model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=3)




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=b5e7be6fb0246e3a4e7f76c06820095265b6772d0ed8da20206df8b17e8fe122
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency reso

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from google.colab import files
uploaded = files.upload()


Saving labeled_telegram_subset.conll to labeled_telegram_subset.conll


In [3]:
# Display the uploaded files to ensure the file is there
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')


Uploaded file: labeled_telegram_subset.conll


In [4]:
import pandas as pd

# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
print(f'Loading file: {filename}')

# Initialize lists to hold tokens and labels
tokens = []
labels = []

# Read the file line by line
with open(filename, 'r') as file:
    for line in file:
        line = line.strip()
        if line:  # Skip empty lines
            parts = line.split(maxsplit=1)  # Split by the first space only
            if len(parts) == 2:  # Ensure there are exactly 2 parts
                token, label = parts
                tokens.append(token)
                labels.append(label)
            else:
                print(f"Skipping line: {line}")  # Debugging output for lines that don't match the format

# Create a DataFrame
df = pd.DataFrame({'Token': tokens, 'Label': labels})

# Display the first few rows of the DataFrame to ensure it was loaded correctly
print(df.head(10))


Loading file: labeled_telegram_subset.conll
                                 Token      Label
0  ...................................          O
1                             Electric          O
2                                 Hair          O
3                              Steamer          O
4                                  Cap          O
5                                 High          O
6                              Quality          O
7                                 የፀጉር          O
8                                እስቲመር  B-PRODUCT
9                               ትሪትመንት          O


In [5]:
# Check unique labels
unique_labels = df['Label'].unique()
print(f"Unique labels: {unique_labels}")


Unique labels: ['O' 'B-PRODUCT' 'B-PRICE' 'I-PRICE' 'B-LOC']


In [6]:
# Create a label mapping
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print(f"Label mapping: {label_mapping}")


Label mapping: {'O': 0, 'B-PRODUCT': 1, 'B-PRICE': 2, 'I-PRICE': 3, 'B-LOC': 4}


In [7]:
# Map the labels in the DataFrame
df['Label'] = df['Label'].map(label_mapping)

# Check for any NaN values after mapping
print("Check for NaN values after mapping:")
print(df[df['Label'].isna()])

# Drop any rows with NaN values after mapping
df.dropna(inplace=True)

# Convert labels to integer type
df['Label'] = df['Label'].astype(int)

# Display the DataFrame after mapping
print(df.head(10))


Check for NaN values after mapping:
Empty DataFrame
Columns: [Token, Label]
Index: []
                                 Token  Label
0  ...................................      0
1                             Electric      0
2                                 Hair      0
3                              Steamer      0
4                                  Cap      0
5                                 High      0
6                              Quality      0
7                                 የፀጉር      0
8                                እስቲመር      1
9                               ትሪትመንት      0


In [8]:
pip install datasets


In [9]:
from transformers import AutoTokenizer
import torch

# Load pre-trained model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['Token'].tolist(),
        padding=True,
        truncation=True,
        is_split_into_words=True,
    )

    labels = []
    word_ids = tokenized_inputs.word_ids(batch_index=0)  # Assuming batch_size is 1
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(int(examples['Label'].values[word_idx]))
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Apply the function to each sentence
tokenized_inputs_list = [tokenize_and_align_labels(sentence) for _, sentence in df.groupby(df.index // 10)]  # Adjust this logic according to your sentence structure

# Convert lists to PyTorch tensors
input_ids = [torch.tensor(item['input_ids'], dtype=torch.long) for item in tokenized_inputs_list]
attention_mask = [torch.tensor(item['attention_mask'], dtype=torch.long) for item in tokenized_inputs_list]
labels = [torch.tensor(item['labels'], dtype=torch.long) for item in tokenized_inputs_list]

# Pad sequences to ensure they have the same length
input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

# Combine into a dataset
from datasets import Dataset

dataset = Dataset.from_dict({
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'labels': labels
})

print(dataset)


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 214
})


In [10]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load the model
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_mapping))

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",  # Change to eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Save the model
model.save_pretrained('./fine-tuned-ner-model')
tokenizer.save_pretrained('./fine-tuned-ner-model')


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,0.334657
2,No log,0.284774
3,No log,0.256856


{'eval_loss': 0.2568563222885132, 'eval_runtime': 65.7369, 'eval_samples_per_second': 3.255, 'eval_steps_per_second': 0.213, 'epoch': 3.0}


('./fine-tuned-ner-model/tokenizer_config.json',
 './fine-tuned-ner-model/special_tokens_map.json',
 './fine-tuned-ner-model/vocab.txt',
 './fine-tuned-ner-model/added_tokens.json',
 './fine-tuned-ner-model/tokenizer.json')